In [115]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer

# Any results you write to the current directory are saved as output.

['imdb_master.csv']


In [116]:
data = pd.read_csv("../input/imdb_master.csv", encoding="ISO-8859-1")

In [117]:
print(data.groupby(['type']).count())

       Unnamed: 0  review  label   file
type                                   
test        25000   25000  25000  25000
train       75000   75000  75000  75000


In [118]:
test_data = data[data.type == "test"]
train_data = data[data.type == "train"][data.label != "unsup"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [119]:
print(len(train_data))

25000


In [120]:
import random
def foo(data):
    train_x = []
    train_y = []
    for i in range(len(data)):
        train_x.append([i.lower() for i in TweetTokenizer().tokenize(data.iloc[i].review) if i.isalpha()])
        label = data.iloc[i].label
        if label == "neg":
            train_y.append(0)
        elif label == "pos":
            train_y.append(1)
    return train_x, train_y

In [121]:
train_x, train_y = foo(train_data)

In [122]:
counter1 = 0
counter0 = 0
c = 0
for i in train_y:
    if i == 1:
        counter1 += 1
    else: counter0 += 1
    c +=1
print(counter0, counter1)

12500 12500


In [123]:
val_x = []
val_y = []
for i in range(3000):
    ind = random.randint(0, len(train_x)-1)
    val_x.append(train_x[ind].copy())
    val_y.append(train_y[ind])
    del train_x[ind]
    del train_y[ind]

In [124]:
c0 = 0
c1 = 0
for i in val_y:
    if i == 0:
        c0 += 1
    else: c1 += 1
print(c0, c1)

1502 1498


In [125]:
test_x, test_y =  foo(test_data)

In [126]:
from collections import Counter

counter = Counter()
for i in train_x:
    for j in i:
        counter[j] += 1


In [127]:
print(counter.most_common(3))

[('the', 296667), ('and', 144573), ('a', 143533)]


In [128]:
print(counter.most_common()[:5])

[('the', 296667), ('and', 144573), ('a', 143533), ('of', 128582), ('to', 119533)]


In [129]:
vocabular = {}
for num, i in enumerate(counter.most_common()):
    vocabular[i[0]] = num+1
    if num > 10000:
        break

In [130]:
print(len(vocabular))

10002


In [131]:
def tokenize(x):
    train_tokenize_x = []
    for i in x:
        temp = []
        for j in i:
            tokenize = vocabular.get(j)
            if tokenize is not None:
                temp.append(tokenize)
        train_tokenize_x.append(temp)
    for i in range(len(train_tokenize_x)):
        if len(train_tokenize_x[i]) < 200:
            train_tokenize_x[i] += [0 for i in range(200-len(train_tokenize_x[i]))]
        else:
            train_tokenize_x[i] = train_tokenize_x[i][:200]
    return train_tokenize_x

In [132]:
train_tokenize_x = tokenize(train_x)
print(train_tokenize_x[3])

[51, 10, 13, 110, 55, 760, 520, 68, 342, 5, 1, 698, 5, 64, 8079, 9, 13, 28, 4, 104, 98, 10, 281, 16, 55, 760, 18, 11, 13, 1, 60, 28, 69, 2479, 44, 4, 222, 89, 10, 65, 109, 106, 8079, 341, 40, 967, 2, 10, 95, 25, 1421, 44, 1, 337, 4, 55, 116, 195, 9, 47, 3, 1812, 9930, 2, 2027, 332, 401, 4, 168, 6492, 2, 3012, 4975, 2197, 1617, 6, 28, 4, 55, 486, 849, 18, 8079, 6, 31, 224, 1, 232, 401, 4, 584, 4, 24, 570, 8, 1, 409, 4, 1617, 383, 169, 3, 444, 9451, 2397, 8, 5, 1, 430, 4, 3, 207, 31, 1, 1819, 3763, 2010, 31, 4168, 570, 111, 821, 1, 19, 4393, 42, 54, 706, 2028, 54, 213, 2, 6, 8, 9030, 6594, 11, 19, 409, 66, 27, 113, 4664, 14, 8, 827, 709, 32, 4049, 4, 396, 5, 3, 49, 9226, 1098, 4, 1201, 2, 833, 18, 2197, 1617, 6, 54, 5152, 1, 19, 6, 2027, 572, 2, 705, 18, 668, 12, 10, 304, 65, 54, 1958, 16, 39, 2293, 15, 96, 4, 1, 100, 280, 10, 392, 60, 5879, 15, 11, 4918, 4, 6212, 8]


In [133]:
val_tokenize_x = tokenize(val_x)
print(val_tokenize_x[1])

[3, 191, 320, 4, 428, 7856, 18, 10, 95, 21, 250, 12, 26, 91, 11, 17, 10, 13, 62, 649, 7, 7, 7856, 204, 1703, 509, 5, 1, 142, 6905, 9, 119, 21, 525, 3, 6667, 1311, 20, 3, 494, 7856, 935, 5, 933, 1124, 186, 1, 100, 46, 6, 54, 186, 7247, 2, 5446, 5446, 46, 23, 924, 96, 135, 16, 49, 70, 103, 79, 8, 1, 2235, 279, 1, 680, 948, 1, 6403, 16, 4241, 6216, 2, 380, 126, 6, 1655, 4241, 6216, 138, 21, 25, 215, 11, 17, 7247, 2, 13, 251, 63, 113, 18, 54, 331, 565, 2, 229, 1123, 115, 360, 1255, 2, 1953, 9, 163, 12, 108, 6, 29, 41, 2717, 2589, 2, 610, 1064, 337, 4, 1, 170, 13, 1846, 6709, 1, 203, 843, 59, 13, 2932, 35, 1, 193, 6905, 752, 1003, 8, 11, 17, 7, 7, 7856, 95, 25, 934, 692, 1, 216, 18, 82, 4, 1, 3182, 400, 1, 17, 26, 4448, 56, 2, 40, 15, 1, 2014, 4, 235, 26, 91, 11, 18, 46, 6, 54, 931, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [134]:
test_tokenize_x = tokenize(test_x)
print(test_tokenize_x[0])

[264, 163, 428, 42, 3118, 44, 3, 17, 15, 224, 1145, 70, 1589, 1150, 35, 1, 1198, 1957, 2168, 793, 4, 59, 46, 23, 50, 158, 10, 40, 115, 21, 439, 41, 96, 4, 1, 100, 87, 4, 169, 25, 2679, 8, 1, 3882, 2, 105, 23, 1622, 378, 20, 2, 89, 1484, 341, 72, 284, 31, 59, 57, 10, 115, 21, 439, 1, 105, 69, 138, 62, 439, 41, 6, 3, 50, 8684, 1, 421, 6, 26, 249, 126, 14, 530, 34, 1276, 121, 70, 241, 309, 179, 85, 2, 266, 54, 3975, 4, 3, 3882, 24, 60, 700, 5, 27, 1741, 122, 397, 51, 69, 23, 71, 479, 1, 356, 92, 206, 4, 11, 3922, 677, 169, 29, 41, 2679, 69, 23, 543, 133, 6, 2638, 5, 27, 1, 113, 16, 54, 2472, 39, 54, 1170, 127, 9, 13, 29, 10, 95, 80, 5, 372, 35, 1527, 9, 126, 32, 531, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [135]:
class Model(torch.nn.Module):
    def __init__(self, v_size):
        super().__init__()
        self.l1 = torch.nn.Embedding(v_size, 50, padding_idx=0)
        self.l2 = torch.nn.LSTM(50, 500, batch_first=True)
        self.l3 = torch.nn.Linear(500, 1)
        
    def forward(self, x):
        out = self.l1(x)
        out, (h_t, h_c) = self.l2(out)
        out = self.l3(h_t)
        return torch.sigmoid(out)

In [136]:
model = Model(len(vocabular)+2).cuda()

In [137]:
class Mydataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __getitem__(self, n):
        return torch.tensor(data=self.x[n]), torch.FloatTensor(data=[self.y[n]])
    
    def __len__(self):
        return len(self.x)

In [138]:
train_dataset = Mydataset(train_tokenize_x, train_y)
train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True)

In [139]:
val_dataset = Mydataset(val_tokenize_x, val_y)
val_dataloader = DataLoader(val_dataset, batch_size=20)

In [140]:
test_dataset = Mydataset(test_tokenize_x, test_y)
test_dataloader = DataLoader(test_dataset, batch_size=20)

In [145]:
optim = torch.optim.Adam(model.parameters(), lr=0.002)
criterion = torch.nn.BCELoss()

In [ ]:
from sklearn.metrics import classification_report
for i in range(7):
    c = 0
    model.train()
    for x, y in train_dataloader:
        optim.zero_grad()
        x, y = x.cuda(), y.cuda()
        out = model(x)[0]
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        if c % 50 == 0:
            print(c/len(train_dataloader))
        c += 1
    model.eval()
    y_labels = []
    y_predicted = []
    with torch.no_grad():
        c = 0
        for x, y in val_dataloader:
            y_labels.extend(y.tolist())
            x, y = x.cuda(), y.cuda()
            out = model(x)[0]
            out_value = []
            for i in out.tolist():
                if i[0] > 0.5:
                    out_value.append(1)
                else:
                    out_value.append(0)
            y_predicted.extend(out_value)
            if c % 50 == 0:
                print(c/len(val_dataloader))
            c += 1
    print(classification_report(y_labels, y_predicted))
        
    
    

0.0
0.045454545454545456
0.09090909090909091
0.13636363636363635
0.18181818181818182
0.22727272727272727
0.2727272727272727
0.3181818181818182
0.36363636363636365
0.4090909090909091
0.45454545454545453
0.5
0.5454545454545454
0.5909090909090909
0.6363636363636364
0.6818181818181818
0.7272727272727273
0.7727272727272727
0.8181818181818182
0.8636363636363636
0.9090909090909091
0.9545454545454546
0.0
0.3333333333333333
0.6666666666666666
              precision    recall  f1-score   support

         0.0       0.53      0.20      0.29      1502
         1.0       0.51      0.82      0.63      1498

   micro avg       0.51      0.51      0.51      3000
   macro avg       0.52      0.51      0.46      3000
weighted avg       0.52      0.51      0.46      3000

0.0
0.045454545454545456
0.09090909090909091
0.13636363636363635
0.18181818181818182
0.22727272727272727
0.2727272727272727
0.3181818181818182
0.36363636363636365
0.4090909090909091
0.45454545454545453
0.5
0.5454545454545454
0.59090909

In [ ]:
from sklearn.metrics import classification_report
y_labels = []
y_predicted = []
with torch.no_grad():
    c = 0
    for x, y in test_dataloader:
        y_labels.extend(y.tolist())
        x, y = x.cuda(), y.cuda()
        out = model(x)[0]
        out_value = []
        for i in out.tolist():
            if i[0] > 0.5:
                out_value.append(1)
            else:
                out_value.append(0)
        y_predicted.extend(out_value)
        if c % 100 == 0:
            print(c/len(test_dataloader))
        c += 1
print(classification_report(y_labels, y_predicted))